In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install Spacy


In [23]:
!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda-autodetect,transformers,lookups]'
!python -m spacy download en_core_web_sm
!python -m spacy download spacy.lang.en.tokenize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached cupy-wheel-11.4.0.tar.gz (3.1 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for cupy-wheel: filename=cupy_wheel-11.4.0-0_cupy-cuda11x-py3-none-any.whl size=1015 sha256=b454fec821d9afcd49f17137987d866e5c80d8991ea51a8a9c36f70235229e4c
  Stored in directory: /root/.cache/pip/wheels/4c/3b/6c/f95d78a37f0358e155b74fa7a9b7c9f52e5f47a42cf38b577d
Failed to build cupy-wheel
  Running setup.py install for cupy-wheel ... done
  DEPRECATION: cupy-wheel was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. pip 23.1 will enforce this behaviour change. A possible replacement is to fix the wheel build issue reported above. Discussion can be found at https://github.com/pypa/pip/issues/8368
/usr/local/lib/python3.8/dist-packages/torch

Import modules

In [6]:
import spacy
import pandas as pd
import nltk
import random
import string
import numpy as np
import seaborn as sns
from spacy.util import minibatch
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, plot_confusion_matrix, classification_report, accuracy_score, f1_score, confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
%matplotlib inline


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [7]:
#load data without metadata

data_1 = pd.read_csv("/content/drive/MyDrive/Uni/emails.csv", delimiter=',')
data_1['text'] = data_1['text'].str.replace('Subject:', '') # remove "Subject: from column"
data_1.head()


,text,spam
0,naturally irresistible your corporate identit...,1
1,the stock trading gunslinger fanny is merril...,1
2,unbelievable new homes made easy im wanting ...,1
3,4 color printing special request additional ...,1
4,"do not have money , get software cds from her...",1


Data exploration

In [8]:

display(data_1.info(),data_1.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5728 non-null   object
 1   spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB


None

,text,spam
0,naturally irresistible your corporate identit...,1
1,the stock trading gunslinger fanny is merril...,1
2,unbelievable new homes made easy im wanting ...,1
3,4 color printing special request additional ...,1
4,"do not have money , get software cds from her...",1


Less spam emails than in the dataset, which creates a heavy bias -> detecting the spam emails will be a chellenge


In [10]:
import plotly.express as px
df= data_1['spam'].value_counts()
fig = px.bar(df,hover_data=[df.index],width=500,height = 300)
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                   marker_line_width=1.5, opacity=0.6)
fig.update_layout(title='Spam 0 or 1')
fig.show()
              

*Data Preprocessing*

1. **Tokenization**

---


---


Tokenization function that takes in a sentence as input and returns a list of tokens (lemmatized, lowercased, and without stop words or punctuation)

In [37]:
#Create list of punctuation marks
punctuations = string.punctuation

# Create list of stopwords
#nlp = spacy.load("en_core_web_sm")
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load the English model and create a new parser object
parser = spacy.load("en_core_web_sm")

# Define the spacy_tokenizer function as before
def spacy_tokenizer(sentence):
    # Tokenize the input sentence using the parser object
    mytokens = parser(sentence)

    # Perform lemmatization and lowercasing
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Remove stop words and punctuations
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # Return the preprocessed list of tokens
    return mytokens


Custom transformer class that takes in a list of texts (X) and returns a list of cleaned texts. The cleaning process consists of stripping leading and trailing spaces and converting the text to lowercase.

In [13]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}
# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [34]:
# creates a CountVectorizer object with the specified tokenizer function
# the output is a matrix of token counts, where each row represents a document and each column represents a token
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))


In [39]:
# creates a TfidfVectorizer object (statistical measure that weights the importance of each word in a document)
# the output is a matrix of TF-IDF features, where each row represents a document and each column represents a token
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

**2. Create Pipeline, train and evaluate models**

---



---



In [16]:
# split into train and test data
X = data_1['text'] # features to analyze
ylabels = data_1['spam'] # labels
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

Create a pipeline with three components: a cleaner, a vectorizer, and a classifier. Then use several algorithm to train the models.

In [38]:
# Logistic Regression Classifier (bow_vector)
classifier = LogisticRegression()

# Create pipeline
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model training
pipe.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f3155382a00>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f315cac91f0>)),
                ('classifier', LogisticRegression())])

In [36]:
# Predicting using the test dataset
predicted = pipe.predict(X_test)


# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted, zero_division=0)) 
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted, zero_division=0))

Logistic Regression Accuracy: 0.9895287958115183
Logistic Regression Precision: 0.9749430523917996
Logistic Regression Recall: 0.9839080459770115


In [40]:
# Random Forest Classifier (bow_vector)
classifier_1 = classifier = RandomForestClassifier(random_state=42)

# Create pipeline 
pipe_1 = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier_1)])

# model training
pipe_1.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7f31553b5d60>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f315cac91f0>)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
# Predict usning the test dataset
predicted_1 = pipe_1.predict(X_test)


# Model Accuracy
print("Random Forest Accuracy:",metrics.accuracy_score(y_test, predicted_1))
print("Random Forest Precision:",metrics.precision_score(y_test, predicted_1, zero_division=0)) #however using zero_divsion no value in the output!!
print("Random Forest Recall:",metrics.recall_score(y_test, predicted_1, zero_division=0)) #however using zero_divsion no value in the output!!

In [ ]:
# Logistic Regression Classifier (tfidf_vector)
classifier = LogisticRegression()

# Create pipeline 
pipe_2 = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# model training 
pipe_2.fit(X_train,y_train)

In [ ]:
# Predict using the test dataset
predicted_2 = pipe_2.predict(X_test)


# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted_2))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted_2, zero_division=0)) #however using zero_divsion no value in the output!!
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted_2, zero_division=0)) #however using zero_divsion no value in the output!!

In [18]:
# Random Forest Classifier (tfidf_vector)
classifier_1 = classifier = RandomForestClassifier(random_state=42)

# Create pipeline 
pipe_3 = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfidf_vector),
                 ('classifier', classifier_1)])

# model training
pipe_3.fit(X_train,y_train)

ValueError: ignored

In [ ]:
# Predict usning the test dataset
predicted_3 = pipe_3.predict(X_test)


# Model Accuracy
print("Random Forest Accuracy:",metrics.accuracy_score(y_test, predicted_3))
print("Random Forest Precision:",metrics.precision_score(y_test, predicted_3, zero_division=0)) #however using zero_divsion no value in the output!!
print("Random Forest Recall:",metrics.recall_score(y_test, predicted_3, zero_division=0)) #however using zero_divsion no value in the output!!